In [1]:
# Install necessary packages (run these cells if not already installed)
!pip install stanfordcorenlp==3.9.1.1
!pip install torchvision

# Imports
import re
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, T5Tokenizer
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from torch.utils.data import Dataset
from stanfordcorenlp import StanfordCoreNLP
from tqdm import tqdm
import json
import os
import sys
import codecs
import asyncio
import aiofiles

from nltk.stem import PorterStemmer
from scipy.stats import zscore

In [2]:
# Check working directory (optional)
!pwd

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

class MDERank:
    def __init__(self, model_name="bert-base-uncased", pooling="max"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.pooling = pooling

    def compute_embedding(self, text):
        # Chuẩn hóa đầu vào và lấy output từ mô hình BERT
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Lấy hidden_states: [batch_size, sequence_length, hidden_size]
        hidden_states = outputs.last_hidden_state[0]  # (seq_len, hidden_size)
        if self.pooling == "max":
            embedding, _ = torch.max(hidden_states, dim=0)
        elif self.pooling == "avg":
            embedding = torch.mean(hidden_states, dim=0)
        else:
            embedding = torch.mean(hidden_states, dim=0)
        return embedding.numpy()

    def extract_candidates(self, text):
        """
        Sử dụng NLTK để tách từ, gán nhãn POS và trích xuất các cụm từ ứng viên
        theo pattern: liên tục các từ có tag bắt đầu bằng JJ (tính từ) hoặc NN (danh từ).
        """
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        candidates = []
        candidate = []
        for word, tag in tagged:
            if tag.startswith("JJ") or tag.startswith("NN"):
                candidate.append(word)
            else:
                if candidate:
                    phrase = " ".join(candidate)
                    candidates.append(phrase)
                    candidate = []
        if candidate:
            phrase = " ".join(candidate)
            candidates.append(phrase)
        # Loại bỏ các cụm từ trùng lặp và có độ dài ít nhất 1 từ
        candidates = list(set([c for c in candidates if len(c.split()) >= 1]))
        return candidates

    def mask_text(self, text, candidate):
        """
        Thay thế các xuất hiện của candidate trong text bằng [MASK] với số lượng token tương ứng.
        """
        candidate_tokens = candidate.split()
        mask_token = " ".join(["[MASK]"] * len(candidate_tokens))
        # Sử dụng regex để thay thế, không phân biệt hoa thường
        pattern = re.compile(re.escape(candidate), re.IGNORECASE)
        masked_text = pattern.sub(mask_token, text)
        return masked_text

    def cosine_similarity(self, vec1, vec2):
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-8)

    def rank_keyphrases(self, text):
        """
        Tính toán embedding của văn bản gốc và đối với mỗi ứng viên, tính embedding của văn bản đã mask.
        Sau đó, tính cosine similarity giữa hai embedding này. Ứng viên có similarity thấp hơn (nghĩa là mất thông tin lớn)
        được xem là quan trọng hơn.
        """
        original_embedding = self.compute_embedding(text)
        candidates = self.extract_candidates(text)
        scores = {}
        for candidate in candidates:
            masked_text = self.mask_text(text, candidate)
            masked_embedding = self.compute_embedding(masked_text)
            sim = self.cosine_similarity(original_embedding, masked_embedding)
            scores[candidate] = sim
        # Sắp xếp các ứng viên theo thứ tự tăng dần của similarity

        results = [(k, v) for k, v in scores.items()]
        keyphrases = [kw[0] for kw in results]
        scores = [kw[1] for kw in results]
        z_score = zscore(scores)
        selected_keyphrases = [(kw, z) for kw, z in zip(keyphrases, z_score) if z <= 0.5]

        # Sắp xếp các ứng viên theo cosine similarity tăng dần (ứng viên có mất thông tin lớn hơn có giá trị similarity thấp hơn)
        ranked = sorted(selected_keyphrases, key=lambda x: x[1])
        return ranked


class AsyncMDERank:
    def __init__(self, model_name="bert-base-uncased", pooling="max"):
        """
        Khởi tạo lớp AsyncMDERank với model BERT và phương pháp pooling.

        Args:
            model_name (str): Tên của model BERT.
            pooling (str): Phương pháp pooling, mặc định là "max". Các tùy chọn khác có thể là "avg".
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.pooling = pooling

    async def compute_embedding(self, text):
        """
        Tính toán embedding của văn bản sử dụng model BERT một cách bất đồng bộ.

        Args:
            text (str): Văn bản đầu vào.

        Returns:
            numpy.ndarray: Embedding của văn bản dưới dạng mảng numpy.
        """
        def _compute():
            # Chuẩn hóa đầu vào và lấy output từ model BERT
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Lấy hidden_states: [seq_len, hidden_size]
            hidden_states = outputs.last_hidden_state[0]
            if self.pooling == "max":
                embedding, _ = torch.max(hidden_states, dim=0)
            elif self.pooling == "avg":
                embedding = torch.mean(hidden_states, dim=0)
            else:
                embedding = torch.mean(hidden_states, dim=0)
            return embedding.numpy()
        
        return await asyncio.to_thread(_compute)

    async def extract_candidates(self, text):
        """
        Sử dụng NLTK để tách từ, gán nhãn POS và trích xuất các cụm từ ứng viên dựa theo pattern:
        các từ liên tiếp có tag bắt đầu bằng 'JJ' (tính từ) hoặc 'NN' (danh từ).

        Args:
            text (str): Văn bản đầu vào.

        Returns:
            list[str]: Danh sách các cụm từ ứng viên (đã loại bỏ trùng lặp).
        """
        def _extract():
            tokens = word_tokenize(text)
            tagged = pos_tag(tokens)
            candidates = []
            candidate = []
            for word, tag in tagged:
                if tag.startswith("JJ") or tag.startswith("NN"):
                    candidate.append(word)
                else:
                    if candidate:
                        phrase = " ".join(candidate)
                        candidates.append(phrase)
                        candidate = []
            if candidate:
                phrase = " ".join(candidate)
                candidates.append(phrase)
            # Loại bỏ các cụm từ trùng lặp và chỉ giữ lại cụm có ít nhất 1 từ
            candidates = list(set([c for c in candidates if len(c.split()) >= 1]))
            return candidates
        
        return await asyncio.to_thread(_extract)

    def mask_text(self, text, candidate):
        """
        Thay thế tất cả các xuất hiện của candidate trong text bằng [MASK] với số lượng token tương ứng.

        Args:
            text (str): Văn bản gốc.
            candidate (str): Cụm từ cần mask.

        Returns:
            str: Văn bản sau khi đã mask.
        """
        candidate_tokens = candidate.split()
        mask_token = " ".join(["[MASK]"] * len(candidate_tokens))
        pattern = re.compile(re.escape(candidate), re.IGNORECASE)
        masked_text = pattern.sub(mask_token, text)
        return masked_text

    def cosine_similarity(self, vec1, vec2):
        """
        Tính cosine similarity giữa hai vector.

        Args:
            vec1 (numpy.ndarray): Vector thứ nhất.
            vec2 (numpy.ndarray): Vector thứ hai.

        Returns:
            float: Giá trị cosine similarity.
        """
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-8)

    async def rank_keyphrases(self, text):
        """
        Tính toán embedding của văn bản gốc và đối với mỗi ứng viên, tính embedding của văn bản đã mask.
        Sau đó, tính cosine similarity giữa hai embedding này. Các ứng viên có similarity thấp hơn (nghĩa là mất
        thông tin lớn) được xem là quan trọng hơn.

        Args:
            text (str): Văn bản đầu vào.

        Returns:
            list[tuple[str, float]]: Danh sách các cặp (ứng viên, similarity) được sắp xếp theo thứ tự tăng dần của similarity.
        """
        # Tính embedding cho văn bản gốc và trích xuất các ứng viên bất đồng bộ
        original_embedding = await self.compute_embedding(text)
        candidates = await self.extract_candidates(text)

        async def process_candidate(candidate):
            masked_text = self.mask_text(text, candidate)
            masked_embedding = await self.compute_embedding(masked_text)
            sim = self.cosine_similarity(original_embedding, masked_embedding)
            return candidate, sim

        # Chạy đồng thời tính toán cho tất cả các ứng viên
        tasks = [process_candidate(candidate) for candidate in candidates]
        results = await asyncio.gather(*tasks)
        keyphrases = [kw[0] for kw in results]
        scores = [kw[1] for kw in results]
        z_score = zscore(scores)
        selected_keyphrases = [(kw, z) for kw, z in zip(keyphrases, z_score) if z <= 0.5]

        # Sắp xếp các ứng viên theo cosine similarity tăng dần (ứng viên có mất thông tin lớn hơn có giá trị similarity thấp hơn)
        ranked = sorted(selected_keyphrases, key=lambda x: x[1])

        return ranked

/kaggle/working
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
async def extract_keyphrases(text, top_k=50):
    """
    Hàm bọc để trích xuất keyphrase từ văn bản.
    Trả về danh sách top_k keyphrase có score thấp nhất (nghĩa là quan trọng nhất).
    """
    mde = AsyncMDERank()
    text = text.lower()
    ranked = await mde.rank_keyphrases(text)
    top_candidates = [phrase for phrase, score in ranked[:top_k]]
    return top_candidates

In [4]:
def clean_labels(labels):
    clean_labels = {}
    for id in labels:
        label = labels[id]
        clean_label = []
        for kp in label:
            if kp.find(";") != -1:
                left, right = kp.split(";")
                clean_label.append(left)
                clean_label.append(right)
            else:
                clean_label.append(kp)
        clean_labels[id] = clean_label        
    return clean_labels

# --------------------
# Các hàm đọc file bất đồng bộ
# --------------------

async def get_long_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/nus/nus_test.json"):
    """ Load file.jsonl bất đồng bộ """
    data = {}
    labels = {}
    async with aiofiles.open(file_path, 'r', encoding='utf-8') as f:
        json_text = await f.readlines()
        for i, line in tqdm(enumerate(json_text), desc="Loading Doc ..."):
            try:
                jsonl = json.loads(line)
                keywords = jsonl['keywords'].lower().split(";")
                abstract = jsonl['abstract']
                doc = abstract
                doc = re.sub(r'\. ', ' . ', doc)
                doc = re.sub(r', ', ' , ', doc)
                doc = doc.replace('\n', ' ')
                data[jsonl['name']] = doc
                labels[jsonl['name']] = keywords
            except Exception as e:
                raise ValueError(f"Lỗi xử lý dòng {i}: {e}")
    labels = clean_labels(labels)
    return data, labels

async def get_duc2001_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/DUC2001"):
    pattern = re.compile(r'<TEXT>(.*?)</TEXT>', re.S)
    data = {}
    labels = {}
    for dirname, _, filenames in os.walk(file_path):
        for fname in filenames:
            if fname == "annotations.txt":
                infile = os.path.join(dirname, fname)
                async with aiofiles.open(infile, 'rb') as f:
                    text = await f.read()
                    text = text.decode('utf8')
                lines = text.splitlines()
                for line in lines:
                    left, right = line.split("@")
                    d = right.split(";")[:-1]
                    l = left
                    labels[l] = d
            else:
                infile = os.path.join(dirname, fname)
                async with aiofiles.open(infile, 'rb') as f:
                    text = await f.read()
                    text = text.decode('utf8')
                found = re.findall(pattern, text)
                if found:
                    data[fname] = found[0]
    labels = clean_labels(labels)
    return data, labels

async def get_inspec_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/Inspec"):
    data = {}
    labels = {}
    for dirname, _, filenames in os.walk(file_path):
        for fname in filenames:
            left, right = fname.split('.')
            infile = os.path.join(dirname, fname)
            if right == "abstr":
                async with aiofiles.open(infile, 'r', encoding='utf-8') as f:
                    text = await f.read()
                    text = text.replace("%", '')
                    data[left] = text
            elif right == "uncontr":
                async with aiofiles.open(infile, 'r', encoding='utf-8') as f:
                    text = await f.read()
                    text = text.replace("\n\t", ' ')
                    text = text.replace("\n", ' ')
                    label = text.split("; ")
                    labels[left] = label
    labels = clean_labels(labels)
    return data, labels

async def get_semeval2017_data(data_path="/kaggle/input/keypharses-extraction-dataset/data/SemEval2017/docsutf8", labels_path="/kaggle/input/keypharses-extraction-dataset/data/SemEval2017/keys"):
    data = {}
    labels = {}
    for dirname, _, filenames in os.walk(data_path):
        for fname in filenames:
            left, right = fname.split('.')
            infile = os.path.join(dirname, fname)
            async with aiofiles.open(infile, "r", encoding="utf-8") as fi:
                text = await fi.read()
                text = text.replace("%", '')
            data[left] = text.lower()
    for dirname, _, filenames in os.walk(labels_path):
        for fname in filenames:
            left, right = fname.split('.')
            infile = os.path.join(dirname, fname)
            async with aiofiles.open(infile, 'r', encoding='utf-8') as f:
                text = await f.read()
                text = text.strip()
                ls = text.splitlines()
                labels[left] = ls
    labels = clean_labels(labels)
    return data, labels

async def get_short_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/krapivin/kravipin_test.json"):
    data = {}
    labels = {}
    async with aiofiles.open(file_path, 'r', encoding='utf-8') as f:
        json_text = await f.readlines()
        for i, line in tqdm(enumerate(json_text), desc="Loading Doc ..."):
            try:
                jsonl = json.loads(line)
                keywords = jsonl['keywords'].lower().split(";")
                abstract = jsonl['abstract']
                doc = abstract
                doc = re.sub(r'\. ', ' . ', doc)
                doc = re.sub(r', ', ' , ', doc)
                doc = doc.replace('\n', ' ')
                doc = doc.replace('\t', ' ')
                data[i] = doc
                labels[i] = keywords
            except Exception as e:
                raise ValueError(f"Lỗi xử lý dòng {i}: {e}")
    labels = clean_labels(labels)
    return data, labels

# Các hàm đơn giản chỉ gọi hàm đã định nghĩa trên
async def get_krapivin_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/krapivin/krapivin_test.json"):
    return await get_short_data(file_path)

async def get_nus_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/nus/nus_test.json"):
    return await get_long_data(file_path)

async def get_semeval2010_data(file_path="/kaggle/input/keypharses-extraction-dataset/data/SemEval2010/semeval_test.json"):
    return await get_short_data(file_path)

async def get_dataset_data(dataset_name):
    if dataset_name == "duc2001":
        return await get_duc2001_data()
    elif dataset_name == "inspec":
        return await get_inspec_data()
    elif dataset_name == "krapivin":
        return await get_krapivin_data()
    elif dataset_name == "nus":
        return await get_nus_data()
    elif dataset_name == "semeval2010":
        return await get_semeval2010_data()
    elif dataset_name == "sameval2017":
        return await get_semeval2017_data()

# Hàm tính F1 (không cần async vì chỉ tính toán)
def calculate_f1(keyphrases: list, ground_truth: list) -> float:
    """
    Tính F1 score cho keyphrases dự đoán so với danh sách keyphrases thực tế.

    Args:
        keyphrases (list): Danh sách keyphrases dự đoán.
        ground_truth (list): Danh sách keyphrases thực tế (labels[id]).

    Returns:
        float: F1 score dưới dạng phần trăm.
    """
    # Tìm các keyphrase chung giữa dự đoán và thực tế
    common = set(keyphrases) & set(ground_truth)
    
    # Tính precision và recall
    precision = len(common) / len(keyphrases) if keyphrases else 0
    recall = len(common) / len(ground_truth) if ground_truth else 0
    
    # Nếu không có giá trị nào thì F1 = 0
    if precision + recall == 0:
        return 0.0
    
    # Tính F1 score và chuyển đổi sang phần trăm
    f1 = 2 * precision * recall / (precision + recall)
    return f1 * 100

# Hàm chuyển cụm từ về dạng stem
def stem_phrase(phrase: str) -> str:
    """
    Chuyển đổi một cụm từ sang dạng stem (dạng gốc).
    Tách cụm từ thành từng từ, stem từng từ rồi nối lại theo thứ tự ban đầu.
    
    Args:
        phrase (str): Cụm từ đầu vào.
    
    Returns:
        str: Cụm từ sau stem.
    """
    stemmer = PorterStemmer()
    tokens = phrase.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)

# Hàm loại bỏ trùng lặp (deduplication)
def dedup(input_list: list) -> list:
    seen = set()
    result = []
    for item in input_list:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

def dedup_stem(input_list: list) -> list:
    return dedup([stem_phrase(item) for item in input_list])

# Hàm ghi kết quả ra file JSON (bất đồng bộ)
def print_to_json(data_name, score):
    """
    Ghi kết quả đánh giá ra file JSON.
    
    Parameters:
      data_name (str): Tên dataset.
      score (list): Danh sách kết quả đánh giá.
    """
    average_score = sum(score) / len(score) if score else 0
    result = {
        "dataset": data_name,
        "average_score": average_score,
    }

    # os.makedirs("data/results/", exist_ok=True)
    with open(f"/kaggle/working/{data_name}.json", "w") as outfile:
        json.dump(result, outfile)

# Hàm main bất đồng bộ
async def main():
    # dataset = ['duc2001', 'inspec', 'krapivin', 'nus', 'semeval2010', 'sameval2017']
    dataset = [ 'duc2001']
    
    for data_name in dataset:
        data, labels = await get_dataset_data(data_name)
        scores = []
        cnt = 0
        for id in data:
            keyphrases = await extract_keyphrases(data[id])
            labels[id] = dedup_stem(labels[id])
            keyphrases = dedup_stem(keyphrases)
            
            print(f"**{data_name}** ", id, " : --> ", cnt, " / ", len(data))
            cnt += 1
            # print(data[id])
            # print("--- labels: ", labels[id])
            # print("--- keyphrases: ", keyphrases)
            score = calculate_f1(keyphrases, labels[id])
            scores.append(score)
            # print("F1 score:", score)
        print_to_json(data_name, scores)

if __name__ == "__main__":
    await main()
    print("Done")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

**duc2001**  LA012090-0090  : -->  0  /  307
**duc2001**  WSJ900720-0113  : -->  1  /  307
**duc2001**  AP881009-0072  : -->  2  /  307
**duc2001**  WSJ920103-0037  : -->  3  /  307
**duc2001**  AP880913-0129  : -->  4  /  307
**duc2001**  FT933-10881  : -->  5  /  307
**duc2001**  AP901029-0035  : -->  8  /  307
**duc2001**  AP880816-0234  : -->  9  /  307
**duc2001**  AP891210-0079  : -->  10  /  307
**duc2001**  AP880630-0295  : -->  11  /  307
**duc2001**  AP900619-0006  : -->  12  /  307
**duc2001**  WSJ900914-0127  : -->  13  /  307
**duc2001**  AP901012-0032  : -->  14  /  307
**duc2001**  FT932-12322  : -->  15  /  307
**duc2001**  AP880705-0109  : -->  16  /  307
**duc2001**  LA071590-0068  : -->  17  /  307
**duc2001**  FT922-3171  : -->  18  /  307
**duc2001**  FT911-2650  : -->  19  /  307
**duc2001**  AP901203-0166  : -->  20  /  307
**duc2001**  AP890117-0132  : -->  21  /  307
**duc2001**  LA110590-0038  : -->  22  /  307
**duc2001**  FT923-5835  : -->  23  /  307
**duc2